In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler

# Load the training data
train_data = pd.read_csv('/content/drive/MyDrive/train.csv')

# Display class distribution
print("Class distribution in training data:")
print(train_data['project_is_approved'].value_counts(normalize=True))

# Preprocess function to handle both train and test data
def preprocess_data(data):
    # Handling missing values
    data['teacher_prefix'].fillna(data['teacher_prefix'].mode()[0], inplace=True)
    # Drop columns that won't be used in the model
    data.drop(columns=['project_essay_3', 'project_essay_4', 'teacher_id', 'project_title', 'project_essay_1', 'project_essay_2', 'project_resource_summary'], inplace=True)
    # Convert 'project_submitted_datetime' to datetime and extract features
    data['project_submitted_datetime'] = pd.to_datetime(data['project_submitted_datetime'])
    data['submission_month'] = data['project_submitted_datetime'].dt.month
    data['submission_day'] = data['project_submitted_datetime'].dt.day
    data['submission_hour'] = data['project_submitted_datetime'].dt.hour
    data.drop('project_submitted_datetime', axis=1, inplace=True)
    # One-hot encoding
    categorical_cols = ['teacher_prefix', 'school_state', 'project_grade_category', 'project_subject_categories', 'project_subject_subcategories']
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
    return data

# Process training data
train_data_processed = preprocess_data(train_data.copy())

# Separate features and target
X_train = train_data_processed.drop('project_is_approved', axis=1)
y_train = train_data_processed['project_is_approved']

# Oversampling to handle class imbalance
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Train a logistic regression model with class weight balanced
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_resampled, y_resampled)

# Load and process the test data
test_data = pd.read_csv('/content/drive/MyDrive/test.csv')
test_data_processed = preprocess_data(test_data.copy())
# Align test data features with training data features
test_data_processed = test_data_processed.reindex(columns=X_train.columns, fill_value=0)

# Predict on the test set
test_predictions = model.predict(test_data_processed)

# Save predictions to CSV file
submission = pd.DataFrame(test_predictions, columns=['predicted_label'])

submission.to_csv('/content/drive/MyDrive/Predict_SE22UARI117', index=False)

# Evaluate model
y_val_pred = model.predict(X_train)
print("Confusion Matrix:\n", confusion_matrix(y_train, y_val_pred))
print("Classification Report:\n", classification_report(y_train, y_val_pred))


Class distribution in training data:
project_is_approved
1    0.849184
0    0.150816
Name: proportion, dtype: float64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Confusion Matrix:
 [[ 7652  5529]
 [29480 44737]]
Classification Report:
               precision    recall  f1-score   support

           0       0.21      0.58      0.30     13181
           1       0.89      0.60      0.72     74217

    accuracy                           0.60     87398
   macro avg       0.55      0.59      0.51     87398
weighted avg       0.79      0.60      0.66     87398

